In [ ]:
# Databricks notebook source
import mlflow
from mlflow.tracking import MlflowClient
import sys
import os

# ====================== CONFIGURATION ========================= #
try:
    EXPERIMENT_NAME = dbutils.widgets.get("experiment_name")
    print(f"✓ Experiment Name from widget: {EXPERIMENT_NAME}")
except:
    EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Delta_XGBoost"
    print(f"ℹ Using default experiment: {EXPERIMENT_NAME}")

UC_CATALOG = "workspace"
UC_SCHEMA = "ml"

# =================== MODEL CONFIG METADATA ==================== #
MODEL_CONFIG = {
    "xgboost": {
        "model_name": "house_price_xgboost_uc2",
        "artifact_path": "xgboost_model",
        "param_keys": [
            "best_n_estimators", "best_max_depth", "best_learning_rate",
            "best_subsample", "best_colsample_bytree"
        ],
        "metric_key": "test_rmse",
        "keywords": ["xgboost", "xgb"]
    }
}

# ================== MODEL TYPE DETECTION ====================== #
def detect_model_config(experiment_name: str):
    exp_lower = experiment_name.lower()
    for model_type, cfg in MODEL_CONFIG.items():
        for key in cfg["keywords"]:
            if key in exp_lower:
                full_uc_name = f"{UC_CATALOG}.{UC_SCHEMA}.{cfg['model_name']}"
                print(f"✓ Detected model type: {model_type.upper()}")
                print(f"✓ UC Model Name: {full_uc_name}")
                return (
                    full_uc_name,
                    cfg["artifact_path"],
                    cfg["param_keys"],
                    cfg["metric_key"]
                )
    raise ValueError("❌ No matching model config found based on experiment name!")

REGISTERED_MODEL_NAME, ARTIFACT_PATH, PARAM_KEYS, METRIC_KEY = detect_model_config(EXPERIMENT_NAME)
TOL = 1e-6  # float tolerance

# ====================== UTILITIES ====================== #
def normalize(val):
    try:
        if '.' not in str(val) and str(val).isdigit():
            return int(val)
        return float(val)
    except:
        return str(val)

# ================== FIND BEST RUN ====================== #
def get_best_run(client):
    exp = client.get_experiment_by_name(EXPERIMENT_NAME)
    if not exp:
        print("❌ Experiment not found.")
        return None, {}, {}

    # fetch all runs from this experiment
    runs = client.search_runs([exp.experiment_id], order_by=["start_time DESC"], max_results=50)
    if not runs:
        print("⚠ No runs found in experiment.")
        return None, {}, {}

    best_run = None
    best_metric = float("inf")

    for r in runs:
        metric_val = r.data.metrics.get(METRIC_KEY)
        if metric_val is not None and metric_val < best_metric:
            best_metric = metric_val
            best_run = r

    if best_run:
        params = {k: normalize(v) for k, v in best_run.data.params.items() if k in PARAM_KEYS}
        metrics = best_run.data.metrics
        print(f"🏆 Best Run Found: {best_run.info.run_id}")
        print(f"   {METRIC_KEY} = {best_metric}")
        print("   Params:", params)
        return best_run.info.run_id, params, metrics
    else:
        print("⚠ No valid runs with metric found.")
        return None, {}, {}

# ================ DUPLICATE VERSION CHECK ===================== #
def check_duplicate(client, new_params, new_metrics):
    mv_list = client.search_model_versions(f"name = '{REGISTERED_MODEL_NAME}'")
    if not mv_list:
        return None

    new_metric_val = new_metrics.get(METRIC_KEY, None)
    for mv in mv_list:
        try:
            run = client.get_run(mv.run_id)
        except Exception:
            continue

        old_params = {k: normalize(v) for k, v in run.data.params.items() if k in new_params}
        old_metric_val = run.data.metrics.get(METRIC_KEY, None)

        same_params = all(old_params.get(k) == new_params.get(k) for k in new_params)
        same_metric = (
            old_metric_val is not None and new_metric_val is not None
            and abs(old_metric_val - new_metric_val) <= TOL
        )
        if same_params and same_metric:
            print(f"\n⏭️ Duplicate Model Found! Version {mv.version} already has same params & metric.")
            return mv
    return None

# ================== REGISTER MODEL LOGIC ======================= #
def register_model(client, run_id, params, metrics):
    duplicate_version = check_duplicate(client, params, metrics)
    if duplicate_version:
        print(f"✅ Using existing registered version: {duplicate_version.version}")
        return duplicate_version

    model_uri = f"runs:/{run_id}/{ARTIFACT_PATH}"
    print(f"⏳ Registering new model from: {model_uri}")

    try:
        new_version = mlflow.register_model(model_uri, REGISTERED_MODEL_NAME)
        print("\n✅ Model Registered Successfully!")
        print(f"   Name: {REGISTERED_MODEL_NAME}")
        print(f"   Version: {new_version.version}")

        # 🏷️ Add helpful tags for tracking
        client.set_model_version_tag(REGISTERED_MODEL_NAME, new_version.version, "source_run_id", run_id)
        client.set_model_version_tag(REGISTERED_MODEL_NAME, new_version.version, "experiment_name", EXPERIMENT_NAME)
        client.set_model_version_tag(REGISTERED_MODEL_NAME, new_version.version, "metric_rmse", str(metrics.get(METRIC_KEY, "")))

        return new_version
    except Exception as e:
        print("❌ Registration Failed:", e)
        sys.exit(1)

# ============================ MAIN ============================ #
if __name__ == "__main__":
    print("\n" + "=" * 70)
    print("🚀 MODEL REGISTRATION WITH BEST RUN SELECTION (MLflow + UC)")
    print("=" * 70 + "\n")

    client = MlflowClient()
    run_id, params, metrics = get_best_run(client)

    if run_id:
        register_model(client, run_id, params, metrics)
    else:
        print("❌ No valid best run found. Exiting.")
